In [1]:
# #MDVR RGB Split


# import os
# import shutil
# import pandas as pd
# from sklearn.model_selection import GroupShuffleSplit

# # Configuration
# # SOURCE_ROOT = r"C:\NPersonal\Projects\SDP\Prediction Stuff\Dataset\MDVR Spectrogram Dataset"
# # DEST_ROOT = r"C:\NPersonal\Projects\SDP\Prediction Stuff\Dataset\MDVR"

# SOURCE_ROOT = r"/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR RGB Spectrogram Dataset"
# DEST_ROOT = r"/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR RGB"

# SPLIT_RATIO = (0.7, 0.15, 0.15)  # Train, Val, Test
# SEED = 42

# def extract_patient_number(folder_name):
#     """Extract base patient number from complex folder names"""
#     # Example: "S_ID00_hc_0_0_0" -> "ID00"
#     # Example: "R_PD_123_1_2" -> "PD_123"
#     parts = folder_name.split('_')
#     if parts[0] in ['S', 'R']:
#         base_parts = parts[1:-3]  # Remove S/R prefix and numeric suffixes
#     else:
#         base_parts = parts[:-3]  # For non-S/R prefixed patients
#     return '_'.join(base_parts)

# def process_dataset():
#     # Collect all patients with their base numbers
#     patients = []
    
#     for class_name in ['HC', 'PD']:
#         class_path = os.path.join(SOURCE_ROOT, class_name)
#         if not os.path.exists(class_path):
#             continue
            
#         for folder in os.listdir(class_path):
#             folder_path = os.path.join(class_path, folder)
#             if os.path.isdir(folder_path):
#                 base_number = extract_patient_number(folder)
#                 patients.append({
#                     'original_path': folder_path,
#                     'class': class_name,
#                     'base_number': base_number,
#                     'full_name': folder
#                 })

#     # Create DataFrame and group by base number
#     df = pd.DataFrame(patients)
#     grouped = df.groupby(['base_number', 'class'])

#     # Prepare data for splitting
#     unique_patients = []
#     for (base_num, cls), group in grouped:
#         unique_patients.append({
#             'base_number': base_num,
#             'class': cls,
#             'paths': group['original_path'].tolist(),
#             'count': len(group)
#         })

#     # Convert to DataFrame for splitting
#     split_df = pd.DataFrame(unique_patients)

#     # Split patients into train/val/test
#     gss = GroupShuffleSplit(n_splits=1, test_size=SPLIT_RATIO[1]+SPLIT_RATIO[2], random_state=SEED)
#     train_idx, temp_idx = next(gss.split(split_df, groups=split_df['base_number']))

#     gss_val_test = GroupShuffleSplit(n_splits=1, test_size=SPLIT_RATIO[2]/(SPLIT_RATIO[1]+SPLIT_RATIO[2]), random_state=SEED)
#     val_idx, test_idx = next(gss_val_test.split(split_df.iloc[temp_idx], groups=split_df.iloc[temp_idx]['base_number']))

#     # Create splits
#     splits = {
#         'train': split_df.iloc[train_idx],
#         'val': split_df.iloc[temp_idx].iloc[val_idx],
#         'test': split_df.iloc[temp_idx].iloc[test_idx]
#     }

#     # Copy files while preserving S/R versions
#     for split_name, split_data in splits.items():
#         print(f"Processing {split_name} set...")
#         for _, patient in split_data.iterrows():
#             for version_path in patient['paths']:
#                 # Create destination path
#                 dest_folder = os.path.join(
#                     DEST_ROOT,
#                     split_name,
#                     patient['class'],
#                     os.path.basename(version_path)  # Keep original folder name
#                 )
                
#                 os.makedirs(dest_folder, exist_ok=True)
                
#                 # Copy all PNG files
#                 for file in os.listdir(version_path):
#                     if file.lower().endswith('.png'):
#                         src = os.path.join(version_path, file)
#                         dst = os.path.join(dest_folder, file)
#                         if not os.path.exists(dst):
#                             shutil.copy2(src, dst)

#     # Verification
#     print("\n📊 Final Verification:")
#     for split in ['train', 'val', 'test']:
#         split_path = os.path.join(DEST_ROOT, split)
#         total = 0
#         for root, dirs, files in os.walk(split_path):
#             total += len(files)
#         print(f"{split.upper()} - {total} images")
#         print(f"Sample patient folders: {os.listdir(split_path)[:2]}")

# if __name__ == "__main__":
#     process_dataset()
#     print("\n✅ Dataset organized with strict patient-level splits!")

Processing train set...
Processing val set...
Processing test set...

📊 Final Verification:
TRAIN - 41744 images
Sample patient folders: ['HC', 'PD']
VAL - 13509 images
Sample patient folders: ['HC', 'PD']
TEST - 8929 images
Sample patient folders: ['HC', 'PD']

✅ Dataset organized with strict patient-level splits!


In [17]:
# #ITALIAN RGB Split


# import os
# import shutil
# import random

# # Configuration
# SOURCE_ROOT = "/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian Spectrogram 3"
# DEST_ROOT = "/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian RGB 3"

# SPLIT_RATIO = (0.7, 0.10, 0.10)  # Train, Val, Test
# random.seed(35)

# # Step 1: Collect patient data
# patients = {"HC": [], "PD": []}

# for class_name in ["HC", "PD"]:
#     class_path = os.path.join(SOURCE_ROOT, class_name)
#     if os.path.exists(class_path):
#         patients[class_name] = [os.path.join(class_path, p) for p in os.listdir(class_path) if os.path.isdir(os.path.join(class_path, p))]

# # Step 2: Split patients into train, val, test
# for class_name, patient_list in patients.items():
#     random.shuffle(patient_list)
#     total = len(patient_list)
    
#     train_split = int(total * SPLIT_RATIO[0])
#     val_split = int(total * SPLIT_RATIO[1])

#     splits = {
#         "train": patient_list[:train_split],
#         "val": patient_list[train_split:train_split + val_split],
#         "test": patient_list[train_split + val_split:]
#     }

#     # ==== NEW: ensure val is the larger of val/test ====
#     if len(splits["test"]) > len(splits["val"]):
#         splits["val"], splits["test"] = splits["test"], splits["val"]

#     # Step 3: Copy files with unique names
#     for split_name, patient_folders in splits.items():
#         print(f"Processing {split_name} for {class_name}...")

#         for patient_folder in patient_folders:
#             patient_name = os.path.basename(patient_folder).replace(" ", "_")  # Ensure safe folder names
#             dest_patient_folder = os.path.join(DEST_ROOT, split_name, class_name, patient_name)
#             os.makedirs(dest_patient_folder, exist_ok=True)

#             for session_folder in os.listdir(patient_folder):
#                 session_path = os.path.join(patient_folder, session_folder)
#                 if os.path.isdir(session_path):
#                     # Iterate through images and rename uniquely
#                     for idx, file in enumerate(sorted(os.listdir(session_path)), start=1):
#                         if file.lower().endswith('.png'):
#                             new_filename = f"{session_folder}_{idx}.png"  # Unique name based on session
#                             src = os.path.join(session_path, file)
#                             dst = os.path.join(dest_patient_folder, new_filename)
#                             shutil.copy2(src, dst)

# # Step 4: Verification
# for split in ["train", "val", "test"]:
#     split_path = os.path.join(DEST_ROOT, split)
#     total = sum([len(files) for _, _, files in os.walk(split_path)])
#     print(f"{split.upper()} - {total} images")

# print("\n✅ Italian dataset split successfully with unique filenames!")

Processing train for HC...
Processing val for HC...
Processing test for HC...
Processing train for PD...
Processing val for PD...
Processing test for PD...
TRAIN - 82983 images
VAL - 22311 images
TEST - 7972 images

✅ Italian dataset split successfully with unique filenames!


In [4]:
import os
import shutil
import random

# Configuration
SOURCE_ROOT = "/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian Spectrogram 30hopLength"
DEST_ROOT = "/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian RGB 3"

SPLIT_RATIO = (0.7, 0.15, 0.15)  # Train, Val, Test
random.seed(40)

# Step 1: Collect patient data
patients = {"HC": [], "PD": []}

for class_name in ["HC", "PD"]:
    class_path = os.path.join(SOURCE_ROOT, class_name)
    if os.path.exists(class_path):
        patients[class_name] = [os.path.join(class_path, p) for p in os.listdir(class_path) if os.path.isdir(os.path.join(class_path, p))]

# Helper functions
def patient_image_count(patient_folder):
    count = 0
    for session_folder in os.listdir(patient_folder):
        session_path = os.path.join(patient_folder, session_folder)
        if os.path.isdir(session_path):
            count += len([f for f in os.listdir(session_path) if f.lower().endswith('.png')])
    return count

def total_images(patient_folders):
    return sum([patient_image_count(p) for p in patient_folders])

# Step 2: Initial random split (by patient counts)
splits = {"train": [], "val": [], "test": []}

for class_name, patient_list in patients.items():
    random.shuffle(patient_list)
    total = len(patient_list)
    train_split = int(total * SPLIT_RATIO[0])
    val_split = int(total * SPLIT_RATIO[1])

    class_splits = {
        "train": patient_list[:train_split],
        "val": patient_list[train_split:train_split + val_split],
        "test": patient_list[train_split + val_split:]
    }

    for split_name, patient_folders in class_splits.items():
        splits[split_name].extend(patient_folders)

# Step 3: Refine splits based on actual image counts
all_patients = splits["train"] + splits["val"] + splits["test"]
ideal_total = total_images(all_patients)

ideal_counts = {
    "train": ideal_total * SPLIT_RATIO[0],
    "val": ideal_total * SPLIT_RATIO[1],
    "test": ideal_total * SPLIT_RATIO[2],
}

# Continue shifting until close enough
def current_counts(splits):
    return {k: total_images(v) for k,v in splits.items()}

def find_best_patient_to_move(from_split, to_split):
    best_diff = float('inf')
    best_idx = None
    for idx, patient in enumerate(splits[from_split]):
        patient_imgs = patient_image_count(patient)
        new_from = current[from_split] - patient_imgs
        new_to = current[to_split] + patient_imgs
        diff = abs(new_from - ideal_counts[from_split]) + abs(new_to - ideal_counts[to_split])
        if diff < best_diff:
            best_diff = diff
            best_idx = idx
    return best_idx

# Keep refining
threshold = 0.01 * ideal_total  # 1% tolerance
current = current_counts(splits)

while True:
    diffs = {k: current[k] - ideal_counts[k] for k in splits.keys()}
    too_big = [k for k in diffs if diffs[k] > threshold]
    too_small = [k for k in diffs if diffs[k] < -threshold]

    if not too_big or not too_small:
        break  # Done

    from_split = max(too_big, key=lambda k: diffs[k])
    to_split = min(too_small, key=lambda k: diffs[k])

    best_idx = find_best_patient_to_move(from_split, to_split)
    if best_idx is None:
        break

    patient = splits[from_split].pop(best_idx)
    splits[to_split].append(patient)

    current = current_counts(splits)

# Step 4: Copy files with unique names
for split_name, patient_folders in splits.items():
    print(f"Processing {split_name} with {len(patient_folders)} patients...")

    for patient_folder in patient_folders:
        patient_name = os.path.basename(patient_folder).replace(" ", "_")
        dest_patient_folder = os.path.join(DEST_ROOT, split_name, os.path.basename(os.path.dirname(patient_folder)), patient_name)
        os.makedirs(dest_patient_folder, exist_ok=True)

        for session_folder in os.listdir(patient_folder):
            session_path = os.path.join(patient_folder, session_folder)
            if os.path.isdir(session_path):
                for idx, file in enumerate(sorted(os.listdir(session_path)), start=1):
                    if file.lower().endswith('.png'):
                        new_filename = f"{session_folder}_{idx}.png"
                        src = os.path.join(session_path, file)
                        dst = os.path.join(dest_patient_folder, new_filename)
                        shutil.copy2(src, dst)

# Step 5: Verification
for split in ["train", "val", "test"]:
    split_path = os.path.join(DEST_ROOT, split)
    total = sum([len(files) for _, _, files in os.walk(split_path)])
    print(f"{split.upper()} - {total} images")

print("\n✅ Italian dataset split successfully with as even image counts as possible!")


Processing train with 41 patients...
Processing val with 10 patients...
Processing test with 10 patients...
TRAIN - 79613 images
VAL - 16792 images
TEST - 16861 images

✅ Italian dataset split successfully with as even image counts as possible!
